In [14]:
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import BDay
import requests
import sys
import numpy as np
%pylab inline
import re

sys.path.append('D:/Python/AntoTradingSystem/Lib/')
import yafin

master = yafin.getmaster('Stock')
masterindex = yafin.getmaster('Index')

from pandas_datareader.data import DataReader


Populating the interactive namespace from numpy and matplotlib


In [16]:
DataReader('MCF', 'yahoo', '2010/10/1', '2015/3/1').head(2)

In [21]:
yahoo.get_timeseries_gross_return(list(masterindex.head(3).Ticker), '2016/10/1', '2017/3/1').head()

,24794573.DE,^NQUSS9572GBPT,^NQEMEA9000EURT
2016-10-03,NaN,NaN,NaN
2016-10-04,NaN,NaN,NaN
2016-10-05,NaN,NaN,NaN
2016-10-06,NaN,NaN,NaN
2016-10-07,NaN,NaN,NaN


In [8]:
sys.path.append('D:/Python/AntoTradingSystem/Lib/backtest-master/')
import calculator as cal
from calculator import yahoo

symbols = list(master.head(8).Ticker)
startdate = dt.date(2015,10,1)
enddate = dt.date(2016,6,1)

yahoo.get_timeseries_gross_return(symbols, '2010/10/1', '2015/3/1').head()

,VATSAEDU.BO,RES.TO,WELLNESS.BO,WNGRF,ECV.L,RXNRP,HAL,MCF
2010-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#put 8 company portfolio
#need to incorporat check so that 1 stock can't be twice in the same date range
import sqlite3

loc = 'D:/Python/AntoTradingSystem/db/'

conn = sqlite3.connect(loc + '/portfoliosim.sqlite')
cur = conn.cursor()

reset_db()

#symbols = list(master.head(8).Ticker)
symbols = list(['GOOG', 'SAN.MC', 'BBVA.MC', 'TSLA', 'NFLX', 'PEP', 'CSCO', 'PSX'])

lst_vf = ['2010-10-01', '2010-10-01', '2010-10-15', '2010-11-01', '2010-11-01', '2010-10-01', '2011-02-01', '2011-02-15']
lst_vt = ['2012-05-01', '2012-11-01', '2012-12-15', '2011-03-01', '2011-10-01', '2010-12-01', '2011-05-01', '2011-11-15']
units = [1000, 1000, 100, 500, 1000, 200, 1000, 500]

i = 0
for symbol in symbols:
    add_comp2pf(2, np.nan , symbol, '', units[i], lst_vf[i], lst_vt[i])
    i = i + 1
    print(symbol)

In [ ]:
stocks_2 = ['GOOG', 'SAN.MC', 'BBVA.MC', 'TSLA', 'NFLX', 'PEP', 'CSCO', 'PSX']

In [ ]:
gticker='NYSE:IBM'
import pandas.io.data as web
dfg = web.DataReader(gticker, 'google', '2013/1/1', '2014/3/1')


In [ ]:
yafin.find_stock(master, 'phillips')

In [ ]:
def reset_db():
    cur.executescript('''
    DROP TABLE IF EXISTS pfDef;
    DROP TABLE IF EXISTS pfComp;
    
    CREATE TABLE pfDef (
        pf_id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name   TEXT UNIQUE,
        description TEXT,
        inception DATE,
        history DATE
    );

    CREATE TABLE pfComp (
        pf_id     INTEGER,
        comp_id   INTEGER,
        yf_id TEXT,
        comp_name TEXT,
        units INTEGER,
        validfrom DATE,
        validto DATE,
        PRIMARY KEY (pf_id, comp_id)
    )
    ''')
    
def add_comp2pf(pf_id, comp_id, yf_id, comp_name, units, validfrom, validto):

    cur.execute('''INSERT OR IGNORE INTO pfComp (pf_id, comp_id, yf_id, comp_name, units, validfrom, validto) 
        VALUES ( ?, ?, ?, ?, ?, ?, ? )''', ( pf_id, comp_id, yf_id, comp_name, units, validfrom, validto, ) )

    conn.commit()

In [ ]:
#get composition for portfolio
cur.execute('SELECT * FROM pfComp WHERE pf_id = ? ', (2, ))

df1 = pd.DataFrame(cur.fetchall(), columns=[des[0] for des in cur.description])
df1

In [ ]:
symbols = list(['GOOG', 'SAN.MC', 'BBVA.MC', 'TSLA', 'NFLX', 'PEP', 'CSCO', 'PSX'])

In [ ]:
#develop calcualtor
calc_from = '2010-10-01'
calc_to = '2012-12-15'

pr = yahoo.get_timeseries_gross_return(symbols, calc_from, calc_to)
pr = pr.fillna(method='bfill')
pr.index.name = None
#pr = pr / pr.iloc[0]
pr.head()

In [ ]:
symbols = list(['GOOG', 'SAN.MC', 'BBVA.MC', 'TSLA', 'NFLX', 'PEP', 'CSCO', 'PSX'])
pdreb
uniquedates

In [ ]:
#get composition for portfolio
port_id = 2
cur.execute('SELECT * FROM pfComp WHERE pf_id = ? ', (port_id, ))

df1 = pd.DataFrame(cur.fetchall(), columns=[des[0] for des in cur.description])

#get unique portfolios and calculate index
data = []
basevalue = 1000
uniquedates = sorted(df1.validfrom.drop_duplicates())[:]
for d in range(len(uniquedates)):
    
    vf = uniquedates[d]
    try:
        vt = uniquedates[d+1]
    except:
        vt = str(dt.date.today())
    cur.execute('SELECT * FROM pfComp WHERE pf_id = ? and validfrom <= ? and validto >= ?', (port_id, vf, vf,))
    pdreb = pd.DataFrame(cur.fetchall(), columns=[des[0] for des in cur.description]) #portfolio at rebalancing
    print(d, vf, vt)
    
    symbols = list(pdreb.yf_id)
    units = list(pdreb.units)
    pr = yahoo.get_timeseries_gross_return(symbols, vf, vt).fillna(method='bfill')
    pr.index.name = None
    pr = pr / pr.iloc[0] #standardize
    
    prreb = list(pr.iloc[0,:])
    mcap = np.array(prreb) * np.array(units)
    wgts = [(i/sum(mcap[mcap>=0])) for i in mcap]
    
    for s in range(len(symbols)):
        pr[symbols[s]] = pr[symbols[s]] * wgts[s]
    
    portfolio = pr.sum(axis=1)
    portfolio = portfolio / portfolio.iloc[0] * basevalue
    data.append(portfolio)
    
    print(portfolio[-1])
    basevalue = portfolio[-1]
    
portfolio = pd.DataFrame(pd.concat(data)).drop_duplicates()
portfolio.columns = ['TR']
    
print('done')
portfolio.plot()
portfolio.head()

In [ ]:
import os
import re
import pandas as pd
from pandas.tseries.offsets import BDay #pandas has holiday functionality (tbd)
import datetime as dt
from backtest.externaldata.yahoo import get_timeseries_gross_return

def calculate(folderloc, portfoliosymbol):
    '''Return portfolio value history
    
    filenames have to be of the format '{portfoliosymbol}_{YYYYMMDD}.csv',
    the date in the filename is the effective date, i.e. weights are
    open-weights; the files need to be comma-separated and include at
    least the columns 'Symbol' and 'Weight'    
    
    Keyword arguments:
    folderloc -- path to folder that contains composition files (string)
    indexsymbol -- indexsymbol of index to be calculated
    '''
    basevalue = 100
    data = []
    
    datelist = sorted([pd.to_datetime(re.split('[\_\.]',x)[1],format='%Y%m%d').date() for x in os.listdir(folderloc) if x.startswith(portfoliosymbol+'_')])
    
    for filedate in datelist: 
        # define dates
        try:
            nextfiledate = [x for x in datelist if x > filedate][0]
        except:
            nextfiledate = dt.date.today()

        # read composition to dataframe
        fileloc = folderloc + '/' + portfoliosymbol + '_' + dt.datetime.strftime(filedate,'%Y%m%d') + '.csv'
        comp = pd.read_csv(fileloc, dtype={'Symbol':str})
        
        # load total return indices from QAD DataStream tables
        returns = get_timeseries_gross_return(list(comp['Symbol']),(filedate - BDay(1)).date(),(nextfiledate - BDay(1)).date()).fillna(method='ffill')

        # standardize
        returns = returns / returns.iloc[0]
        # apply weights
        for index, row in comp.iterrows():
            returns[row['Symbol']] = returns[row['Symbol']] * row['Weight']
        # sum up to get index timeseries
        portfolio = returns.sum(axis=1)
        # scale to match previous indexvalue
        portfolio = portfolio / portfolio.iloc[0] * basevalue
        # patch together
        data.append(portfolio)
        basevalue = portfolio[-1]

    portfolio = pd.DataFrame(pd.concat(data)).drop_duplicates()
    portfolio.columns = [portfoliosymbol+'_TR']

    return portfolio